# This file is to take the .xlsx file created by reach_Curator_py38_v3.py script, and adds in the missing rows for T6000, T5000 values
- has multiple steps
1. Backs up the raw .xlsx file created by Reach_Curator_py38_v3.py
2. Takes raw .xlsx file created by Reach_Curator_py38_v3.py --> Adds in missing rows by reading _events.txt file and comparing it --> puts this modifed version with added rows in the root folder Grant_curate
3. Once you have manually curated the session (ie. added reaches) using the Reach_Curator_py38_v3.py
4. this will back up that manually curated .xlsx file into this folder, which it creates if it does not exist already --> completed_manual_curation_backup
5. finally, it will remove any rows that have no values for ReachInit > ReachMax > ReachEnd_ --> it will then put this final cleaned version back into the root folder that the Reach_Curator_py38_v3.py saves and reads from --> Grant_curate

# ⭐ What this file does ⭐

## ✅ Results of running this file
1. you have created a duplicate of the .xlsx file that the reach_curator_py38_v3.py makes when you load in a session for the first time 
    - Backup location --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/reach_curator_direct_backup
    - Note, this file is missing all the T6000 and T5000 rows that the find_reach_events.ipynb did not associate a reach too
2. you have created a new .xlsx file that now containes every sinlge row, meaning it added rows for every single T6000 > T5000, and left the reachInit > reachMax > reachEnd vlaues empty, this way you can manually go into the curator and add reaches for these rows
    - Backup location --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/added_missing_rows_backup

    
### ✅ Next Steps after running this file
1. Go back to reach_Curator_p38_v3.py, and simply launch it liuke normal. then manually place all the reaches you need 

2. Once 100% done adding reachs and label. open the analyze_curated_reach_results.ipynb file (its in the same folder as this notebook)
    - then simply run that file
    - it will first back up your manaual curation .xlsx file too --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/completed_manual_curation_backup
    - it will extract / drop all empty rows from that .xlsx file and save the final version too -->  Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/final_backup
    - and it will save this final_df to the root folder, so the reach_curator can load it in --> Grant_curate/final_xlsx_file.xlsx 

In [1]:
import pandas as pd
import numpy as np
import os
import shutil


## Modify these three variables as needed

In [2]:
root_path = r'G:\Grant\behavior_data\DLC_net' # change this to your root path where all the analysis folder live, i currently use is G:\Grant\behavior_data\DLC_net
root_folder = 'grant_reach10_swingDoor-christie' # change this to your root folder for one specfic mouse (this should contain multiple sessions)


# Set the specific mouse session to analyze
session_options = ['20250421_session001', '20250424_session001', '20250426_session001','20250429_session003','20250501_session001']

session_info = session_options[-1] # change this to the session you want to analyze
print(session_info)

20250501_session001


### Create new session_option for each session
- class that lets you load data for specific mouse sesssion

In [3]:
main_path = rf'{root_path}\{root_folder}'
print(f'main_path: {main_path}')

class select_mouse_session:
    def __init__(self, mouse, session_date, session_ID):
        self.mouse = mouse
        self.session_date = session_date
        self.session_ID = session_ID

    def __repr__(self):
        return f"Mouse: {self.mouse}, Session Date: {self.session_date}, Session ID: {self.session_ID}"
    
    def get_mouse_session_files(self):
        mouse_session = f"{self.mouse}_{self.session_date}_{self.session_ID}"
        xlsx_file = rf'{main_path}\Grant_curate\{self.session_date}_christielab_{self.session_ID}.xlsx'
        txt_file = rf"{main_path}\videos\{self.session_date}\christielab\{self.session_ID}\{self.session_date}_christielab_{self.session_ID}_events_shifted.txt"
        xlsx_orig = rf'{main_path}\Grant_curate\xlsx_backups\curator_direct_backup\{self.session_date}_christielab_{self.session_ID}.xlsx'
        
        return xlsx_file, txt_file, xlsx_orig
    
    def get_mouse_info(self):
        mouse_info = {
            'mouse': self.mouse,
            'session_date': self.session_date,
            'session_ID': self.session_ID
        }
        return mouse_info
    

# Strip the session info to get the session date and ID
session_date = session_info.split('_')[0]
session_ID = session_info.split('_')[1]

# Strip the mouse name from the root folder
mouse = root_folder.split('_')[1]

# Create an instance of the class with the mouse name, session date, and session ID
class_intsance = select_mouse_session(mouse='reach10', session_date=session_date, session_ID=session_ID)

# Get the mouse session files and info
xlsx_file, txt_file, xlsx_orig = class_intsance.get_mouse_session_files()
mouse_info = class_intsance.get_mouse_info()

# Print the results
print(f'mouse_info: {mouse_info}')
print(f'xlsx_file: {xlsx_file}')

main_path: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie
mouse_info: {'mouse': 'reach10', 'session_date': '20250501', 'session_ID': 'session001'}
xlsx_file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\20250501_christielab_session001.xlsx


In [4]:
session_date = mouse_info['session_date']
session_ID = mouse_info['session_ID']
mouse = mouse_info['mouse']
session_save_name = f"{mouse}_{session_date}_{session_ID}"
print('=========================')
print(f'session_save_name: {session_save_name}')
print('')
print(f'mouse: {mouse}')
print(f'session_date: {session_date}')
print(f'session_ID: {session_ID}')


    # ---- Step 1: Make a backup in curator_backup folder
original_dir = os.path.dirname(xlsx_file)
backup_dir = os.path.join(original_dir, 'xlsx_backups',session_save_name,'reach_curator_direct_backup')
os.makedirs(backup_dir, exist_ok=True)
print(f'original_dir: {original_dir}')
print(f'backup_dir: {backup_dir}')


session_save_name: reach10_20250501_session001

mouse: reach10
session_date: 20250501
session_ID: session001
original_dir: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate
backup_dir: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\xlsx_backups\reach10_20250501_session001\reach_curator_direct_backup


## Checking if you have already analyzed this session
- if you have, the code is going to stop you from re-analyzing it by KILLING the keneral
- you you do want to overwrite these files --> you need to re-run from the top and skip this cell

In [5]:
final_df_backup = os.path.join(os.path.dirname(xlsx_file), 'xlsx_backups',session_save_name ,'final_backup')
final_df_backup_file_path = os.path.join(final_df_backup, os.path.basename(xlsx_file))

manual_curation_backup = os.path.join(os.path.dirname(xlsx_file),'xlsx_backups', session_save_name,'completed_manual_curation_backup')
manual_curation_backup = os.path.join(manual_curation_backup, os.path.basename(xlsx_file))

if os.path.exists(manual_curation_backup):
    print('⚠️ Looks like you have already completed the manual curation for this session ⚠️')
    print('❌ Therefore, in an effort to prevent overwriting any files, the script changed is going to KILL the kernal (sorry)❌')
    print('')
    print(f'manual_curation_backup: {manual_curation_backup}')
    print('')
    print('===========================================')
    print(' ✅ if you actually want to run this script on this session then simply skip running this cell next time ✅')
    print('===========================================')
    exit()

print('')
if os.path.exists(final_df_backup_file_path):
    print('⚠️ Looks like you have already completed the final curation for this session ⚠️')
    print('❌ Therefore, in an effort to prevent overwriting any files, the script changed is going to KILL the kernal (sorry)❌')
    print('')
    print(f'final_df_backup_file_path: {final_df_backup_file_path}')
    print('')
    print('===========================================')
    print(' ✅ if you actually want to run this script on this session then simply skip running this cell next time ✅')
    print('===========================================')    
    exit()



## Verify existence of session files

In [7]:


if not os.path.exists(txt_file):
    print(f"⚠️Warning: {txt_file} does not exist⚠️")

backup_file_path = os.path.join(backup_dir, os.path.basename(xlsx_file))
if os.path.exists(backup_file_path):
    print(f"⚠️Warning: Backup file already exists at {backup_file_path}⚠️")
    print('')
else:
        # Check if files exist
    if not os.path.exists(xlsx_file):
        print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
    else:
        # Copy the original file to the backup directory
        shutil.copy2(xlsx_file, backup_file_path)
        print(f"✅ Backup saved to: {backup_file_path}")

# ---- (your parsing & merging code goes here)
# ... your code to build `combined` DataFrame ...

print(f'backup_dir: {backup_file_path}')
print('=========================')
print(f'xlsx_file: {xlsx_file}')
print('=========================')
print(f'txt_file: {txt_file}')
print('=========================')
print(f'xlsx_orig: {xlsx_orig}')


⚠️Warning: Backup file already exists at G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\xlsx_backups\reach10_20250501_session001\reach_curator_direct_backup\20250501_christielab_session001.xlsx⚠️

backup_dir: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\xlsx_backups\reach10_20250501_session001\reach_curator_direct_backup\20250501_christielab_session001.xlsx
xlsx_file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\20250501_christielab_session001.xlsx
txt_file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\videos\20250501\christielab\session001\20250501_christielab_session001_events_shifted.txt
xlsx_orig: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\xlsx_backups\curator_direct_backup\20250501_christielab_session001.xlsx


### This is the cell that takes the .xlsx file and the .txt file and creates a new .xlsx file with the manual additions
- makes a new folder called manual_additions if it does not exist
- reads the .xlsx file into a pandas dataframe
- reads the .txt file into a pandas dataframe
- creates a new dataframe with the manual additions
- saves the new dataframe to a new .xlsx file



In [8]:
# Load existing curated xlsx
print(f"Loading existing xlsx file: {xlsx_file}")
df_orig = pd.read_excel(xlsx_file)
df_orig.head()

Loading existing xlsx file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\20250501_christielab_session001.xlsx


,reachInit,reachMax,reachEnd,stim,pellet_delivery,pellet_detected,T6000,T5000,Reach_type,behaviors
0,8689,8711,8714,0,8679,8378,8317,8681,0,grabbed
1,17625,17644,17693,0,17604,17300,17228,17607,0,stalled
2,20646,20659,20742,0,20628,20552,20488,20630,0,stalled
3,23112,23132,23135,0,23086,22935,22874,23089,0,missed
4,26169,26206,26267,0,26145,25729,25668,26147,0,stalled


In [9]:

class modify_curator_xlsx_file(object):
    def __init__(self, xlsx_file, txt_file,save_df=True):
        self.txt_file = txt_file
        self.xlsx_file = xlsx_file
        self.save_df = save_df

        
    def create_new_xlxs_file(self):
        save_df = self.save_df
        # Load the existing xlsx file
        df_orig = pd.read_excel(self.xlsx_file)
        # Check if the file is empty
        txt_file = self.txt_file
        if not os.path.exists(txt_file):
            print(f"⚠️Warning: {txt_file} does not exist⚠️")
            return None
        # Check if the xlsx file is empty
        xlsx_file = self.xlsx_file
        if not os.path.exists(xlsx_file):
            print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
            return None

        if df_orig.empty:
            print("⚠️Warning: The xlsx file is empty⚠️")
            return None
        
        # Normalize column names just in case
        df_orig.columns = df_orig.columns.str.strip()

        # Load and parse txt as done earlier
        events = []
        current_event = {}
        with open(txt_file, 'r') as f:
            for line in f:
                if line.strip():
                    key, value = line.strip().split('\t')
                    value = int(value)
                    if key == 'T6000_played':
                        if current_event:
                            events.append(current_event)
                        current_event = {'T6000': value}
                    elif key == 'pellet_detected':
                        current_event['pellet_detected'] = value
                    elif key == 'pellet_delivery':
                        current_event['pellet_delivery'] = value
                    elif key == 'T5000_played':
                        current_event['T5000'] = value
        if current_event:
            events.append(current_event)

        events_df = pd.DataFrame(events)

        # Find which T6000s are missing from original file
        existing_T6000s = df_orig['T6000'].values
        new_rows = events_df[~events_df['T6000'].isin(existing_T6000s)].copy()

        # Fill in missing columns to match structure
        new_rows['reachInit'] = np.nan
        new_rows['reachMax'] = np.nan
        new_rows['reachEnd'] = np.nan
        new_rows['stim'] = 0
        new_rows['behaviors'] = np.nan

        # # Reorder columns to match original file
        # new_rows = new_rows[['reachinit', 'reachmax', 'reachend', 'stim',
        #                      'pellet_delivery', 'pellet_detected', 't6000', 't5000', 'behaviors']]

        # Reorder columns to match original file
        new_rows = new_rows[['T6000', 'T5000', 'reachInit', 'reachMax',
                            'reachEnd', 'stim','behaviors' ,'pellet_delivery', 'pellet_detected']]

        # Combine and sort by T6000
        combined = pd.concat([df_orig, new_rows], ignore_index=True)
        combined = combined.sort_values(by='T6000').reset_index(drop=True)

        # Force final column order
        combined = combined[['T6000', 'T5000', 'reachInit', 'reachMax',
                            'reachEnd', 'stim', 'behaviors',
                            'pellet_delivery', 'pellet_detected']]


        # ---- Step 2: Save output as the same original file (overwrite it)
        xlsx_save_path = xlsx_file
        self.xlsx_file = xlsx_save_path

        original_dir = os.path.dirname(self.xlsx_file)
        backup_dir = os.path.join(original_dir,'xlsx_backups',session_save_name ,'reach_curator_direct_backup')
        backup_file_path = os.path.join(backup_dir, os.path.basename(self.xlsx_file))

        backup_added_rows = os.path.join(original_dir, 'xlsx_backups' ,session_save_name,'added_missing_rows_backup')
        os.makedirs(backup_added_rows, exist_ok=True)
        backup_added_rows_file_path = os.path.join(backup_added_rows, os.path.basename(self.xlsx_file))

        # Save the combined DataFrame to the same xlsx file
        if save_df:
            if os.path.exists(backup_file_path):
                combined.to_excel(xlsx_save_path, index=False)
                combined.to_excel(backup_added_rows_file_path, index=False)
                print(f"✅ Final merged Excel saved to: {xlsx_save_path}")
                print(f"✅ Backup saved to: {backup_added_rows_file_path}")
                return combined, xlsx_save_path
            else:
                print(f"⚠️Warning: Backup file doesnt exists⚠️")
                print('❌ Stopping the script to avoid overwriting the xlsx file without a back up❌')
                print(f"⚠️Warning: {backup_file_path} does not exist⚠️")
                return None, None
        else:
            print(f"⚠️Warning: DataFrame not saved to {xlsx_save_path}⚠️")
            print('set save_df to True to save the DataFrame')
            return combined
    

    def drop_empty_reach_rows(self):
      
        df = pd.read_excel(self.xlsx_file)
        # back up the newly created xlsx file
        backup_dir_02 = os.path.join(os.path.dirname(self.xlsx_file),'xlsx_backups', session_save_name,'completed_manual_curation_backup')
        os.makedirs(backup_dir_02, exist_ok=True)
        backup_file_path_02 = os.path.join(backup_dir_02, os.path.basename(self.xlsx_file))
        if os.path.exists(backup_file_path_02):
            print(f"⚠️Warning: Backup file already exists at {backup_file_path_02}⚠️")
            print('❌ Stopping the script to avoid overwriting the backup file ❌')
            return None 
        else:
            # Copy the original file to the backup directory
            shutil.copy2(self.xlsx_file, backup_file_path_02)
            print(f"✅ Backup saved to: {backup_file_path_02}")
        """
        Drop specified rows from the DataFrame.
        """
        # Drop bad rows
        df = df.dropna(subset=['reachInit']).reset_index(drop=True)
         # save the DataFrame to the same xlsx file
        if self.save_df:
            df.to_excel(self.xlsx_file, index=False)
            print(f"✅ Final merged Excel saved to: {self.xlsx_file}")   
              
        return df
    
    def load_orig_xlsx_file(self):
        """
        Load the original xlsx file.
        """
        original_dir = os.path.dirname(self.xlsx_file)
        backup_dir = os.path.join(original_dir, 'xlsx_backups',session_save_name ,'reach_curator_direct_backup')
        backup_file_path = os.path.join(backup_dir, os.path.basename(self.xlsx_file))

        df_orig = pd.read_excel(backup_file_path)
        return df_orig, backup_file_path
    
    def load_added_rows_xlsx(self):
        """
        Load the added rows xlsx file.
        """
        original_dir = os.path.dirname(self.xlsx_file)
        backup_added_rows = os.path.join(original_dir,'xlsx_backups' ,session_save_name, 'added_missing_rows_backup')
        backup_added_rows_file_path = os.path.join(backup_added_rows, os.path.basename(self.xlsx_file))

        df_added_rows = pd.read_excel(backup_added_rows_file_path)
        return df_added_rows, backup_added_rows_file_path
    
    
    def load_final_xlsx(self):
        """
        Load the combined xlsx file.
        """
        df_final = pd.read_excel(self.xlsx_file)
        return df_final, self.xlsx_file




In [10]:
safe_fail_modify = True

In [11]:
if safe_fail_modify:
    mouse_sessions = modify_curator_xlsx_file(xlsx_file, txt_file,save_df=True)
    df_modied, xlsx_saved_file = mouse_sessions.create_new_xlxs_file()
    df_orig, xlsx_orig_file = mouse_sessions.load_orig_xlsx_file()
    df_added_rows, xlsx_added_rows_file = mouse_sessions.load_added_rows_xlsx()
    df_final, xlsx_final_file = mouse_sessions.load_final_xlsx()
    df_modied
else:
    print(f"⚠️Warning: safe_fail actviated⚠️")
    print('if you want to run the code, please re-run this cell')
    print('Note this ')
    safe_fail_modify = True

✅ Final merged Excel saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\20250501_christielab_session001.xlsx
✅ Backup saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\xlsx_backups\reach10_20250501_session001\added_missing_rows_backup\20250501_christielab_session001.xlsx


In [12]:
# Load existing curated xlsx
df_added_missing_rows = pd.read_excel(xlsx_saved_file)
df_added_missing_rows.head(5)

,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected
0,1805,1952.0,NaN,NaN,NaN,0,NaN,1950.0,1874.0
1,3968,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,5938,6232.0,NaN,NaN,NaN,0,NaN,6230.0,6004.0
3,8317,8681.0,8689.0,8711.0,8714.0,0,grabbed,8679.0,8378.0
4,12505,12725.0,NaN,NaN,NaN,0,NaN,12723.0,12569.0


In [13]:
df_orig, backup_file_path = mouse_sessions.load_orig_xlsx_file()
df_added_missing_rows, backup_added_rows_file_path = mouse_sessions.load_added_rows_xlsx()

In [14]:
orig_reach_count = df_orig.shape[0]
added_row_count = df_added_missing_rows.shape[0]
print(f'loading orginal_df for reach events from --> {backup_file_path}')
print(f'loading df with missing rows added from--> {backup_added_rows_file_path}')
# Count the number of reach events
print('')
print(f"Original reach events: {len(df_orig)}")
print(f"Added reach events: {len(df_added_missing_rows)}")
print('')
print(f'❌ Reaches Missed: {added_row_count- orig_reach_count}')


loading orginal_df for reach events from --> G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\xlsx_backups\reach10_20250501_session001\reach_curator_direct_backup\20250501_christielab_session001.xlsx
loading df with missing rows added from--> G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie\Grant_curate\xlsx_backups\reach10_20250501_session001\added_missing_rows_backup\20250501_christielab_session001.xlsx

Original reach events: 49
Added reach events: 84

❌ Reaches Missed: 35


In [15]:
df_orig.head(15)

,reachInit,reachMax,reachEnd,stim,pellet_delivery,pellet_detected,T6000,T5000,Reach_type,behaviors
0,8689,8711,8714,0,8679,8378,8317,8681,0,grabbed
1,17625,17644,17693,0,17604,17300,17228,17607,0,stalled
2,20646,20659,20742,0,20628,20552,20488,20630,0,stalled
3,23112,23132,23135,0,23086,22935,22874,23089,0,missed
4,26169,26206,26267,0,26145,25729,25668,26147,0,stalled
5,32944,33010,33021,0,32923,32698,32636,32925,0,stalled
6,37480,37495,37558,0,37459,37159,37098,37462,0,stalled
7,43918,43952,44000,0,43898,43486,43421,43901,0,stalled
8,52136,52171,52177,0,52118,51967,51898,52121,0,grabbed
9,54303,54319,54329,0,54275,54197,54132,54277,0,grabbed


In [16]:
df_added_missing_rows.head(15)

,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected
0,1805,1952.0,NaN,NaN,NaN,0,NaN,1950.0,1874.0
1,3968,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,5938,6232.0,NaN,NaN,NaN,0,NaN,6230.0,6004.0
3,8317,8681.0,8689.0,8711.0,8714.0,0,grabbed,8679.0,8378.0
4,12505,12725.0,NaN,NaN,NaN,0,NaN,12723.0,12569.0
5,14767,15062.0,NaN,NaN,NaN,0,NaN,15060.0,14829.0
6,17228,17607.0,17625.0,17644.0,17693.0,0,stalled,17604.0,17300.0
7,20488,20630.0,20646.0,20659.0,20742.0,0,stalled,20628.0,20552.0
8,22874,23089.0,23112.0,23132.0,23135.0,0,missed,23086.0,22935.0
9,25668,26147.0,26169.0,26206.0,26267.0,0,stalled,26145.0,25729.0


# ❌ END ❌


### ✅ Next Steps
1. Go back to reach_Curator_p38_v3.py, and simply launch it liuke normal. then manually place all the reaches you need 

2. Once 100% done adding reachs and label. open the analyze_curated_reach_results.ipynb file (its in the same folder as this notebook)
    - then simply run that file
    - it will first back up your manaual curation .xlsx file too --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/completed_manual_curation_backup
    - it will extract / drop all empty rows from that .xlsx file and save the final version too -->  Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/final_backup
    - and it will save this final_df to the root folder, so the reach_curator can load it in --> Grant_curate/final_xlsx_file.xlsx 


# ✅ Results of file
1. you have created a duplicate of the .xlsx file that the reach_curator_py38_v3.py makes when you load in a session for the first time 
    - Backup location --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/reach_curator_direct_backup
    - Note, this file is missing all the T6000 and T5000 rows that the find_reach_events.ipynb did not associate a reach too
2. you have created a new .xlsx file that now containes every sinlge row, meaning it added rows for every single T6000 > T5000, and left the reachInit > reachMax > reachEnd vlaues empty, this way you can manually go into the curator and add reaches for these rows
    - Backup location --> Grant_curate/xlsx_backups/mouse_sessionDate_sessionID/added_missing_rows_backup
